In [ ]:
pip install tensorflow keras numpy matplotlib opencv-python scikit-learn

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split

In [ ]:
import os
import cv2
import random
import numpy as np

base_path = "TRAIN"
categories = ["O", "R"]
img_size = 64  # Resize to 64x64 for efficiency

data = []
labels = []

for category in categories:
    path = os.path.join(base_path, category)
    class_label = categories.index(category)  # 0 for organic, 1 for recyclable
    
    all_images = os.listdir(path)
    
    # Select 1000 random images per category
    selected_images = random.sample(all_images, min(1000, len(all_images)))

    for img_name in selected_images:
        img_path = os.path.join(path, img_name)
        
        try:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            img = cv2.resize(img, (img_size, img_size))  # Resize
            data.append(img)
            labels.append(class_label)
        except Exception as e:
            print("Error loading image:", img_name, e)

# Convert to NumPy arrays for ML model
data = np.array(data, dtype=np.float32) / 255.0  # Normalize
labels = np.array(labels)

# Shuffle data to mix categories
combined = list(zip(data, labels))
random.shuffle(combined)
data, labels = zip(*combined)
data = np.array(data)
labels = np.array(labels)

print(f"Dataset reduced to {len(data)} images (1000 per category).")
Dataset reduced to 2000 images (1000 per category).

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into Train (80%) and Test (20%)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN model
model = Sequential([
    # Convolutional layers
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    # Flatten for fully connected layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularization
    Dense(2, activation='softmax')  # Output layer (2 classes)
])

# Compile Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
